In [1]:
versioninfo()

Julia Version 1.9.2
Commit e4ee485e909 (2023-07-05 09:39 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 2 on 4 virtual cores


## 15.2 미분 구현 실습

### 구현 1: 손미분

In [2]:
function f(x)
    for _ = 1:2
        x = x*(x-1)
    end
    x
end

f (generic function with 1 method)

### 구현 2: 수치 미분

In [3]:
finite_diff(f, x, h =0.0001) = (f(x + h) -f(x))/h

finite_diff (generic function with 2 methods)

In [4]:
finite_diff(f, 3)

55.003600100143046

### 구현 3: 기호 미분

In [5]:
f_closed_expr = :(x^4 - 2x^3 + x);

In [6]:
f_closed_expr.args

3-element Vector{Any}:
 :+
 :(x ^ 4 - 2 * x ^ 3)
 :x

In [8]:
f_closed_expr.args[2].args

3-element Vector{Any}:
 :-
 :(x ^ 4)
 :(2 * x ^ 3)

각 하위 표현식을 재귀적으로 돌면서 사칙연산 패턴을 찾아서 미분된 표현식을 돌려주는 다음과 같은 함수를 작성한다.

In [10]:
using MacroTools

In [12]:
function derive(ex, x)
    ex == x ? 1 :
    ex isa Union{Number, Symbol} ? 0 :
    @capture(ex, a_ + b_) ? :($(derive(a, x)) + $(derive(b, x))) :
    @capture(ex, a_ - b_) ? :($(derive(a, x)) - $(derive(b, x))) :
    @capture(ex, a_ * b_) ? :($(derive(a, x))*$b + $a*$(derive(b, x))) :
    @capture(ex, a_ / b_) ? :(($(derive(a, x))*$b - $a*$(derive(b, x))) / $b^2) :
    @capture(ex, a_^n_Number) ? :($(derive(a, x)) * ($n * $a^$(n-1))) :
    error("$ex is not differentiable")
end
    

derive (generic function with 1 method)

이 derive 함수는 미분할 함수 $y = f(x)$의 표현식 ex와 독립변수의 심벌 x를 인수로 받는다. 